<a href="https://colab.research.google.com/github/MarianaDuartee/Pyspark/blob/main/Pyspark3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 62.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ce2aaf46bcd614e7033c75c50fba3fce6a2644b8414180eeb930a04fe3853d41
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (SparkSession.builder
         .master("local")
         .appName("aprendendo-dataframes")
         .config("spark.ui.port","4050")
         .getOrCreate())

In [4]:
df = (spark
      .read
      .format("csv")
      .option("header","true")
      .option("inferschema","true")
      .option("delimiter",";")
      .load("/content/drive/MyDrive/DataSets/arquivo_geral.csv")
      )

df.show()

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



---

#Outras Funções - Pyspark


---



#WHEN - OTHERWISE

* Condições para criar ou não novas colunas

* Parecido com IF | ELSE, onde IF = WHEN e ELSE = OTHERWISE

* Exemplo: Criar uma nova coluna com as seguintes condições:

> Se o numero de casos novos for maior que zero, acrescentar o texto "Tem casos Novos"

> Se o numero de casos novos não for maior que zero, acrescentar o texto "Sem casos Novos"

1. O código abaixo cria um novo dataframe (df2) com o dataframe original (df)
2. Além das colunas presentes no df original, cria uma nova coluna chamada status (withColumn("status") ...
3. Os valores passados para essa nova coluna possuem condições especificas (**F.when**(condição , valor passado caso a condição seja True).**otherwise**(valor passado caso a condição seja False)

In [7]:
df2 = (df.withColumn("status",F.when(F.col("casosNovos")>0,"Tem Casos Novos")
       .otherwise("Sem casos novos")))

df2.show(100)

+------+------+----------+----------+---------------+-----------+----------------+---------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|         status|
+------+------+----------+----------+---------------+-----------+----------------+---------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|Sem casos novos|
| Norte|  

* Exemplo: Concatenando a quantidade de casos novos com o status

In [10]:
df3 = (df.withColumn("status", 
                     F.when(F.col('casosNovos') > 0, F.concat(df.casosNovos,F.lit(" casos novos")))
                     .otherwise("Não tem casos novos.")
                     ))

df3.select(F.col('status'),F.col('casosNovos'),F.col('data')).filter(F.col('status') != 'Não tem casos novos.').show()

+--------------+----------+----------+
|        status|casosNovos|      data|
+--------------+----------+----------+
| 1 casos novos|         1|2020-03-20|
| 2 casos novos|         2|2020-03-22|
| 2 casos novos|         2|2020-03-25|
| 1 casos novos|         1|2020-03-27|
| 2 casos novos|         2|2020-03-31|
| 1 casos novos|         1|2020-04-01|
| 1 casos novos|         1|2020-04-02|
| 1 casos novos|         1|2020-04-04|
| 1 casos novos|         1|2020-04-05|
| 2 casos novos|         2|2020-04-06|
| 4 casos novos|         4|2020-04-07|
|10 casos novos|        10|2020-04-09|
| 4 casos novos|         4|2020-04-10|
| 1 casos novos|         1|2020-04-11|
| 2 casos novos|         2|2020-04-12|
| 7 casos novos|         7|2020-04-13|
|22 casos novos|        22|2020-04-14|
| 5 casos novos|         5|2020-04-15|
| 7 casos novos|         7|2020-04-16|
|16 casos novos|        16|2020-04-17|
+--------------+----------+----------+
only showing top 20 rows



#Substring 
* Exige 3 parametros

* Primeiro : A propria string que esta percorrendo

* Segundo : Posição inicial que quer pegar da string

* Terceiro: Tamanho da substring que quer pegar

* F.substring('valor da string', 'Posição inicial', 'Tamanho total')

* Exemplo: Criar colunas com Dia/Mês/Ano para ficar mais organizado no Dataframe

> Isso é feito com a utilização da withColumn que cria uma nova coluna junto com a substring 

> A substring mostra uma parte da string de acordo com os parâmetros escolhidos 

In [17]:
df3 = (df.withColumn("Dia", F.substring(df.data,9,2))
         .withColumn("Mês",F.substring(df.data,6,2))
         .withColumn("Ano",F.substring(df.data,1,4)))

df3.select(F.col('data'),F.col('Dia'),F.col("Mês"),F.col("Ano")).show()

+----------+---+---+----+
|      data|Dia|Mês| Ano|
+----------+---+---+----+
|2020-01-30| 30| 01|2020|
|2020-01-31| 31| 01|2020|
|2020-02-01| 01| 02|2020|
|2020-02-02| 02| 02|2020|
|2020-02-03| 03| 02|2020|
|2020-02-04| 04| 02|2020|
|2020-02-05| 05| 02|2020|
|2020-02-06| 06| 02|2020|
|2020-02-07| 07| 02|2020|
|2020-02-08| 08| 02|2020|
|2020-02-09| 09| 02|2020|
|2020-02-10| 10| 02|2020|
|2020-02-11| 11| 02|2020|
|2020-02-12| 12| 02|2020|
|2020-02-13| 13| 02|2020|
|2020-02-14| 14| 02|2020|
|2020-02-15| 15| 02|2020|
|2020-02-16| 16| 02|2020|
|2020-02-17| 17| 02|2020|
|2020-02-18| 18| 02|2020|
+----------+---+---+----+
only showing top 20 rows

